# cv2学习笔记6
## 色彩直方图
### 绘制色彩直方图
cv2.calcHistcv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate ]]) 
- 第一个参数必须用方括号括起来。
- 第二个参数是用于计算直方图的通道，这里使用灰度图计算直方图，所以就直接使用第一个通道0
- 第三个参数是Mask，这里没有使用，所以用None。
- 第四个参数是histSize，表示这个直方图分成多少份（即多少个直方柱）。
- 第五个参数是表示直方图中各个像素的值，[0.0, 256.0]表示直方图能表示像素值从0.0到256的像素。
- 最后是两个可选参数，由于直方图作为函数结果返回了，所以第六个hist就没有意义了（待确定）
- 最后一个accumulate是一个布尔值，用来表示直方图是否叠加。

In [1]:
import cv2      
import numpy as np      
      
img = cv2.imread("C:/Users/situ.st.1/Pictures/splitman.jpg")      
b, g, r = cv2.split(img)   

   #封装成一个函数                 
def calcAndDrawHist(image, color):    
    hist= cv2.calcHist([image], [0], None, [256], [0.0,255.0])
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(hist)    
    histImg = np.zeros([256,256,3], np.uint8)    
    hpt = int(0.9* 256);    
        
    for h in range(256):    
        intensity = int(hist[h]*hpt/maxVal)    
        cv2.line(histImg,(h,256), (h,256-intensity), color)    
            
    return histImg;   

if __name__ == '__main__':    #主函数
    img = cv2.imread("C:/Users/situ.st.1/Pictures/splitman.jpg")    
    b, g, r = cv2.split(img)    
    
    histImgB = calcAndDrawHist(b, [255, 0, 0])    
    histImgG = calcAndDrawHist(g, [0, 255, 0])    
    histImgR = calcAndDrawHist(r, [0, 0, 255])    
        
    cv2.imshow("histImgB", histImgB)    
    cv2.imshow("histImgG", histImgG)    
    cv2.imshow("histImgR", histImgR)    
    cv2.imshow("Img", img)    
    cv2.waitKey(0)    
    cv2.destroyAllWindows()   

- 同时绘制三个通道的直方图


In [2]:
img = cv2.imread("C:/Users/situ.st.1/Pictures/splitman.jpg") 
h = np.zeros((256,256,3)) #创建用于绘制直方图的全0图像    
         
bins = np.arange(256).reshape(256,1) #直方图中各bin的顶点位置    
color = [ (255,0,0),(0,255,0),(0,0,255) ] #BGR三种颜色   
 
for ch, col in enumerate(color):    #for循环是对三个通道遍历一次，每次绘制相应通道的直方图的折线！
    originHist = cv2.calcHist([img],[ch],None,[256],[0,256])    
    cv2.normalize(originHist, originHist,0,255*0.9,cv2.NORM_MINMAX) #归一化函数。该函数将直方图的范围限定在0-255×0.9之间
    hist=np.int32(np.around(originHist))   #先将生成的原始直方图中的每个元素四舍六入五凑偶取整
    pts = np.column_stack((bins,hist))     #将直方图中每个bin的值转成相应的坐标
    cv2.polylines(h,[pts],False,col) #根据这些点绘制出折线，第三个False参数指出这个折线不需要闭合。第四个参数指定了折线的颜色。
h=np.flipud(h)    #反转绘制好的直方图
         
cv2.imshow('colorhist',h)    
cv2.waitKey(0)   

-1

### 直方图均方化（增强对比度）
在图像处理中，直方图均衡化一般用来均衡图像的强度，或增加图像的对比度。在介绍使用直方图均衡化来拉伸图像的直方图之前，先介绍使用查询表的方法。
观察上图中原始图像的直方图，很容易发现大部分强度值范围都没有用到。因此先检测图像非0的最低（imin）强度值和最高（imax）强度值。
将最低值imin设为0，最高值imax设为255。中间的按255.0*(i-imin)/(imax-imin)+0.5)的形式设置。

In [3]:
image = cv2.imread("C:/Users/situ.st.1/Pictures/caoyuan.jpg", 0) 
cv2.imshow("origin",image); 
lut = np.zeros(256, dtype = image.dtype )#创建空的查找表  
hist= cv2.calcHist([image],[0],None,[256], [0.0,255.0])  
      
minBinNo, maxBinNo = 0, 255  
#计算从左起第一个不为0的直方图位置  
for binNo, binValue in enumerate(hist):  
    if binValue != 0:  
        minBinNo = binNo  
        break  
#计算从右起第一个不为0的直方图位置  
for binNo, binValue in enumerate(reversed(hist)):  
    if binValue != 0:  
        maxBinNo = 255-binNo  
        break  
#print minBinNo, maxBinNo  
  
#生成查找表，方法来自参考文献1第四章第2节  
for i,v in enumerate(lut):  
    print i  
    if i < minBinNo:  
        lut[i] = 0  
    elif i > maxBinNo:  
        lut[i] = 255  
    else:  
        lut[i] = int(255.0*(i-minBinNo)/(maxBinNo-minBinNo)+0.5)  

#计算  
result = cv2.LUT(image, lut)  
cv2.imshow("Result", result)  
cv2.waitKey(0)  
cv2.destroyAllWindows()  

#直方图均衡化之OpenCV函数实现
img = cv2.imread('C:/Users/situ.st.1/Pictures/caoyuan.jpg',0)  
equ = cv2.equalizeHist(img)  
cv2.imshow('equ',equ)  
cv2.waitKey(0)  
cv2.destroyAllWindows() 

1 255
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
